## <span style="color:orange; font-weight:bold">GeoPOINT – Synthetic Point Generator for Geospatial Applications</span>

### <span style="color:red; font-weight:bold">Structure of the GeoPOINT Script</span>

This notebook consists of the following key components:

- **Importing required libraries** for numerical processing, visualization, symbolic computation, and file handling.
&nbsp;
- **User configuration parameters.**  
  These parameters define the number of points, transformation settings, and the characteristics of synthetic noise and geodetic distortion.
&nbsp;
- **Generation of synthetic points** on a spherical surface in the fixed (global) frame.  
  Three variants are supported:
  1. Ideal points (noise-free);  
  2. Points with added random Gaussian noise;  
  3. Points with combined random noise and geodetic distortion model (simulating real-world measurement errors).  
&nbsp;
- **Transformation of point coordinates** into a local (body) frame using parameters defined in the configuration.

- **Export of both coordinate sets** (fixed and local) to Excel (`.xlsx`) files for further analysis or benchmarking.

- **Visualization and statistics** summarizing the generated and transformed data.


### <span style="color:darkgreen; font-weight:bold">Importing Required Libraries</span>

This section imports the Python libraries used throughout the notebook.

- **NumPy** and **pandas** are used for numerical computation and tabular data handling.
- **Matplotlib** (with `mpl_toolkits.mplot3d`) is used for 3D visualization.
- **SymPy** is used for symbolic representation of rotation matrices.
- **JSON** and **os** provide configuration file support and system integration.
- **openpyxl** is used internally by `pandas` for exporting Excel spreadsheets.

All libraries used are open-source and compatible with standard scientific Python environments.


In [ ]:
import sys, site
print(sys.executable)
print(site.getsitepackages())

In [ ]:
import os, json, numpy as np, pandas as pd, matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sympy as sp
from pathlib import Path

try:
    import open3d as o3d
    _HAS_O3D = True
except Exception:
    _HAS_O3D = False

try:
    import scipy, scipy.stats as st
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False

print("Versions → numpy:", np.__version__, "| pandas:", pd.__version__,
      "| matplotlib:", matplotlib.__version__, "| sympy:", sp.__version__,
      "| open3d:", getattr(o3d, "__version__", "n/a"),
      "| scipy:", getattr(scipy, "__version__", "n/a"))

### <span style="color:darkgreen; font-weight:bold">User Configuration Parameters</span>

This dictionary contains all key parameters governing synthetic point generation,  
geodetic error simulation, spatial transformation, and the addition of measurement noise.  
All values can be freely edited in this notebook and reflect typical properties of total stations


In [ ]:
# === Configuration Parameters (editable by the user) ===

config = {
    # --- Point generation (fixed frame) ---
    "npoints": 50,  # Number of synthetic points to generate
    "radius": 50.0,  # Nominal radius of the generated spherical surface [m]

    # Geodetic error model (used for distortion)
    "A": 0.005,      # Distance meter constant error (instrument offset) [m]
    "B": 7.0,        # Distance-dependent error (proportional term) [ppm]
    "angle_noise_arcsec": 1.0,  # Simulated instrument angular precision (standard deviation) [arcseconds]

    # --- Rotation parameters (in grads) ---
    "roll": 5,       # Rotation about X axis (clockwise)
    "pitch": 5,      # Rotation about Y axis (anticlockwise)
    "yaw": 40,       # Rotation about Z axis (clockwise)

    # --- Translation parameters (in meters) ---
    "dx": 10.0,      # Translation along X axis
    "dy": 10.0,      # Translation along Y axis
    "dz": 2.0,       # Translation along Z axis

    # --- Noise added to transformed points (body frame) ---
    "gaussian_std": 0.01,   # Standard deviation of Gaussian noise [m]
    "bias": 0.002          # Constant bias added to all coordinates [m]
}


In [ ]:
# === Input Source Configuration ===
# This section controls the source of input points for GeoPOINT.
#
# Available modes:
#   1. "synthetic"
#      - Uses internally generated synthetic points (default).
#      - Generation parameters are defined in the main `config` dictionary
#        (npoints, radius, Gaussian noise, EDM distortion, etc.).
#
#   2. "pcd"
#      - Loads a point cloud file in .pcd format via Open3D.
#      - If path="auto": automatically fetches a small demo dataset
#        (DemoICPPointClouds from open3d.data).
#      - If path="<your_file>.pcd": loads the given local PCD file.
#      - Options:
#          * downsample_n: randomly downsample to this many points.
#          * center_on_mean: if True, subtract centroid before processing.
#
#   3. "csv"
#      - Loads points from a CSV or TXT file with XYZ columns.
#      - path="<your_file>.csv" must be provided.
#      - `csv` dictionary allows customization:
#          * delimiter: field separator (default ",")
#          * usecols: which columns correspond to X,Y,Z (0-based indices)
#          * skiprows: how many header lines to skip
#
# Other options:
#   - random_seed: controls reproducibility of random downsampling.
#
# Example usage:
#   INPUT_CFG = {"source":"synthetic"}                # synthetic test points
#   INPUT_CFG = {"source":"pcd", "path":"auto"}       # Open3D demo cloud
#   INPUT_CFG = {"source":"pcd", "path":"scan.pcd"}   # load your own PCD
#   INPUT_CFG = {"source":"csv", "path":"points.csv",
#                "csv":{"delimiter":",","usecols":[0,1,2]}}
#
# Note: Only one source is active at a time.

INPUT_CFG = {
    "source": "pcd",                  # "synthetic" | "pcd" | "csv"
    "path": "auto",                   # "auto" uses Open3D sample; or provide a real file path
    "csv": {"delimiter": ",", "usecols": [0,1,2], "skiprows": 0},
    "downsample_n": None,             # e.g. 200_000 for random downsampling
    "center_on_mean": False,          # subtract centroid if True
    "random_seed": 42
}

In [ ]:
# === Real-data auto sample + loaders ===
# This block implements input loaders for different sources:
#
# - "synthetic":
#     Uses synthetic points generated in the notebook and passed as `synthetic_xyz`.
#
# - "pcd":
#     Loads a point cloud in .pcd format using Open3D.
#     * If path="auto": download a small sample (DemoICPPointClouds) via open3d.data.
#     * If path="<your_file>.pcd": load the given local file.
#
# - "csv":
#     Loads XYZ points from a CSV/TXT file.
#     The `cfg["csv"]` dictionary controls delimiter, which columns are used as X,Y,Z,
#     and how many header rows to skip.
#
# Extra options (applied regardless of source):
#   * downsample_n: random subsampling to reduce very dense clouds.
#   * center_on_mean: if True, subtracts centroid to center the point cloud.
#
# Tip for real-data experiments:
#   After loading a real dataset (pcd/csv), you can create a "clean" and "noisy"
#   pair for error/uncertainty validation using `add_real_noise_pair(...)`.
#   This allows the same plotting/validation workflow as for synthetic data.


def _ensure_open3d():
    if not _HAS_O3D:
        raise ImportError("Open3D is required for PCD I/O and auto samples. "
                          "Install with: conda install -c conda-forge open3d")

def _auto_sample_pcd_path() -> Path:
    """Download a tiny real-data sample and return a PCD path."""
    _ensure_open3d()
    # Option 1: classic ICP demo point clouds (couple of .pcd files)
    sample = o3d.data.DemoICPPointClouds()
    return Path(sample.paths[0])  # e.g., cloud_bin_0.pcd

def _load_pcd(path_str: str) -> np.ndarray:
    _ensure_open3d()
    p = Path(path_str)
    if not p.exists():
        raise FileNotFoundError(f"PCD not found: {p}")
    pcd = o3d.io.read_point_cloud(str(p))
    pts = np.asarray(pcd.points, dtype=float)
    if pts.ndim != 2 or pts.shape[1] < 3:
        raise ValueError(f"PCD must contain XYZ; got shape {pts.shape}")
    return pts[:, :3]

def _load_csv_xyz(path_str: str, cfg_csv: dict) -> np.ndarray:
    p = Path(path_str)
    if not p.exists():
        raise FileNotFoundError(f"CSV not found: {p}")
    arr = np.loadtxt(
        p,
        delimiter=cfg_csv.get("delimiter", ","),
        usecols=cfg_csv.get("usecols", [0,1,2]),
        skiprows=cfg_csv.get("skiprows", 0),
    )
    arr = np.atleast_2d(arr).astype(float)
    if arr.shape[1] != 3:
        raise ValueError(f"CSV must have exactly 3 columns (XYZ); got shape {arr.shape}")
    return arr

def load_points_from_source(cfg: dict, synthetic_xyz: np.ndarray | None = None) -> np.ndarray:
    """
    Return XYZ (N,3). If source='synthetic', use provided synthetic_xyz.
    If source='pcd' and path='auto', download a tiny Open3D sample (DemoICPPointClouds).
    If source='csv', read XYZ from CSV according to cfg['csv'].
    Optional random downsampling and centering are applied.
    """
    src = cfg.get("source", "synthetic").lower()
    rng = np.random.default_rng(cfg.get("random_seed", 42))

    if src == "synthetic":
        if synthetic_xyz is None:
            raise ValueError("Pass synthetic_xyz from your generation step.")
        X = synthetic_xyz

    elif src == "pcd":
        path = cfg.get("path", "auto")
        if isinstance(path, str) and path.lower() == "auto":
            p = _auto_sample_pcd_path()
        else:
            p = Path(path)
        X = _load_pcd(str(p))

    elif src == "csv":
        X = _load_csv_xyz(cfg["path"], cfg.get("csv", {}))

    else:
        raise ValueError(f"Unknown source: {src}")

    # Optional random downsampling (useful if the cloud is very dense)
    ds = cfg.get("downsample_n", None)
    if ds is not None and X.shape[0] > ds:
        idx = rng.choice(X.shape[0], size=ds, replace=False)
        X = X[idx]

    # Optional centering
    if cfg.get("center_on_mean", False):
        X = X - X.mean(axis=0, keepdims=True)

    return X

In [ ]:
# Publication-ready plotting (vector export, larger fonts)
import matplotlib as mpl
mpl.rcParams.update({
    "font.size": 14, "axes.titlesize": 16, "axes.labelsize": 14,
    "legend.fontsize": 12, "xtick.labelsize": 12, "ytick.labelsize": 12,
    "figure.dpi": 300
})

def savefig_vec(path: str):
    """Save current Matplotlib figure as vector (SVG/PDF) with tight bbox."""
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    plt.savefig(path, bbox_inches="tight")

### <span style="color:darkgreen; font-weight:bold">Generation of Synthetic Points – Fixed Frame</span>

In this step, synthetic 3D points are generated over a spherical surface to simulate geodetic observations 
in an ideal global coordinate system (fixed frame).

The point generation can apply three different modes of increasing realism:

- **Ideal (noise-free):** applies no disturbance; purely mathematical coordinates, uniformly distributed on the sphere.
- **With random Gaussian noise:** applies stochastic errors by perturbing angles or Cartesian coordinates.
- **With geodetic distortion model:** applies both random noise and systematic errors based on typical properties of a Total Station, 
  including EDM (Electronic Distance Meter) range error parameters (`A`, `B`) and angular precision (in arcseconds) as defined in the `config` dictionary.

The selected variant can be easily changed by switching the called function below.


In [ ]:
def generate_angles(config):
    """
    Generate base spherical angles (theta, phi) for reproducible point sets.
    """
    npoints = config["npoints"]
    theta = np.random.uniform(0, 2 * np.pi, npoints)
    phi = np.arccos(1 - 2 * np.random.uniform(0, 1, npoints))
    return theta, phi


In [ ]:
def uniform_spherical(config, theta=None, phi=None):
    """
    Generate ideal 3D points uniformly distributed on the surface of a sphere,
    representing noise-free geodetic observations from a fixed global reference frame.

    Parameters:
        config (dict): Dictionary containing generation parameters:
            - 'npoints': Number of points to generate.
            - 'radius': Nominal distance from origin [m].
        theta (np.ndarray, optional): Predefined azimuthal angles [rad].
        phi (np.ndarray, optional): Predefined zenith angles [rad].

    Returns:
        points (np.ndarray): 3×N array of Cartesian coordinates [m].
        theta (np.ndarray): Azimuthal angles [rad].
        phi (np.ndarray): Zenith angles [rad].
    """
    npoints = config["npoints"]
    radius = config["radius"]

    # --- Generate or use predefined angles ---
    if theta is None or phi is None:
        theta = np.random.uniform(0, 2 * np.pi, npoints)
        phi = np.arccos(1 - 2 * np.random.uniform(0, 1, npoints))

    # --- Convert spherical to Cartesian coordinates ---
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)

    return np.vstack((x, y, z)), theta, phi


In [ ]:
def uniform_spherical_with_noise(config, theta=None, phi=None):
    """
    Generate 3D points on a spherical surface with added Gaussian noise to angles,
    then convert to Cartesian coordinates and scale by user-defined radius.

    Parameters:
        config (dict): Includes 'npoints', 'radius', and 'angle_noise_arcsec'.
        theta (np.ndarray, optional): Base azimuthal angles [rad] to perturb.
        phi (np.ndarray, optional): Base zenith angles [rad] to perturb.

    Returns:
        points (np.ndarray): 3×N array of noisy 3D coordinates [m]
        theta (np.ndarray): perturbed azimuthal angles [rad]
        phi (np.ndarray): perturbed zenith angles [rad]
    """
    npoints = config["npoints"]
    radius = config["radius"]

    # Arcseconds to radians conversion
    rho = 180 * 3600 / np.pi  # arcseconds per radian
    angle_noise = config["angle_noise_arcsec"] / rho

    # Generate or copy angles
    if theta is None or phi is None:
        theta = np.random.uniform(0, 2 * np.pi, npoints)
        phi = np.arccos(1 - 2 * np.random.uniform(0, 1, npoints))
    else:
        # Ensure correct size
        assert len(theta) == npoints and len(phi) == npoints, "Angle arrays must match npoints"
        theta = theta.copy()
        phi = phi.copy()

    # Add Gaussian angular noise
    theta += angle_noise * np.random.randn(npoints)
    phi += angle_noise * np.random.randn(npoints)

    # Spherical to Cartesian conversion
    x = radius * np.sin(phi) * np.cos(theta)
    y = radius * np.sin(phi) * np.sin(theta)
    z = radius * np.cos(phi)

    return np.vstack((x, y, z)), theta, phi


In [ ]:
# === Build global points (synthetic by default, optionally replaced by real XYZ) ===

# 1) Choose synthetic variant (keep your current choice)
#    - ideal, noise-free:
Xg_3xN, theta, phi = uniform_spherical(config)
#    - or Gaussian angular-noise version:
# Xg_3xN, theta, phi = uniform_spherical_with_noise(config)

# Ensure shape is (N, 3) for downstream steps
Xg_synth = Xg_3xN.T  # from (3, N) -> (N, 3)

# 2) Optional override with real data (PCD/CSV) based on INPUT_CFG
#    If INPUT_CFG["source"] == "synthetic" -> X_global stays synthetic.
#    If "pcd"/"csv" -> loader will read real XYZ and return (N,3).
X_global = load_points_from_source(INPUT_CFG, synthetic_xyz=Xg_synth)

print("X_global shape:", X_global.shape, "| source:", INPUT_CFG["source"])

In [ ]:
def uniform_spherical_with_geodetic_distortion(config, theta=None, phi=None):
    """
    Generate synthetic 3D points using a geodetic error model that includes both angular noise 
    and EDM-based range errors representative of typical total station measurements.

    Parameters:
        config (dict): Configuration dictionary with the following keys:
            - npoints: Number of points to generate
            - radius: Nominal distance from origin [m]
            - A: Constant EDM error [m]
            - B: Proportional EDM error [ppm]
            - angle_noise_arcsec: Angular noise (standard deviation) [arcseconds]
        theta (np.ndarray, optional): Base azimuth angles [rad]
        phi (np.ndarray, optional): Base zenith angles [rad]

    Returns:
        points (np.ndarray): 3×N array of perturbed 3D coordinates [m]
        theta (np.ndarray): Azimuth angles with angular noise [rad]
        phi (np.ndarray): Zenith angles with angular noise [rad]
        R (np.ndarray): Simulated range from origin for each point [m], including EDM errors
    """
    npoints = config["npoints"]
    radius = config["radius"]
    A = config["A"]
    B = config["B"]

    # Arcseconds to radians conversion
    rho = 180 * 3600 / np.pi
    angle_noise = config["angle_noise_arcsec"] / rho

    # Generate or copy base angles
    if theta is None or phi is None:
        theta = np.random.uniform(0, 2 * np.pi, npoints)
        phi = np.arccos(1 - 2 * np.random.uniform(0, 1, npoints))
    else:
        assert len(theta) == npoints and len(phi) == npoints, "Angle arrays must match npoints"
        theta = theta.copy()
        phi = phi.copy()

    # Add angular noise
    theta += angle_noise * np.random.randn(npoints)
    phi += angle_noise * np.random.randn(npoints)

    # Convert to unit direction vectors
    x_unit = np.sin(phi) * np.cos(theta)
    y_unit = np.sin(phi) * np.sin(theta)
    z_unit = np.cos(phi)

    # Ideal range
    S = radius * np.ones_like(x_unit)

    # Add EDM noise
    m_S = A + B * 1e-6 * S  # B in ppm
    R = S + m_S * np.random.randn(npoints)

    # Scale vectors by range
    x = R * x_unit
    y = R * y_unit
    z = R * z_unit

    return np.vstack((x, y, z)), theta, phi, R


### <span style="color:darkgreen; font-weight:bold">Coordinates Transformation – Body Frame</span>

In this step, the synthetic 3D points generated in the fixed (global) frame are transformed  
into a local coordinate system (body frame) defined by user-specified parameters.

The transformation consists of:

- **Translation** along the X, Y, and Z axes;  
- **Rotation** (roll, pitch, yaw), applied in a defined order using standard rotation matrices;  
- Optional **scaling**, if needed, to simulate dimensional adjustments.

Rotation angles are specified in **gradians**, following geodetic conventions,  
and are internally converted to **radians** prior to matrix computation.

This step emulates typical geodetic operations such as aligning local measurements  
(e.g., from a vessel or offshore platform) with a known reference coordinate system.

The transformed coordinates are subsequently made available for export, benchmarking, or further analysis.


In [ ]:
# === Rotation and Translation to Local Frame (Body Frame) ===

# --- Load rotation and translation parameters from config ---
roll = config["roll"]
pitch = config["pitch"]
yaw = config["yaw"]
dx = config["dx"]
dy = config["dy"]
dz = config["dz"]

# --- Convert rotation angles from grads to radians ---
to_rad = np.pi / 200
rx = roll * to_rad
ry = pitch * to_rad
rz = yaw * to_rad

# --- Define symbolic variables for rotation angles ---
rx_sym, ry_sym, rz_sym = sp.symbols('rx ry rz')

# --- Symbolic rotation matrices ---
Rx_sym = sp.Matrix([
    [1, 0, 0],
    [0, sp.cos(rx_sym), sp.sin(rx_sym)],
    [0, -sp.sin(rx_sym), sp.cos(rx_sym)]
])

Ry_sym = sp.Matrix([
    [sp.cos(ry_sym), 0, -sp.sin(ry_sym)],
    [0, 1, 0],
    [sp.sin(ry_sym), 0, sp.cos(ry_sym)]
])

Rz_sym = sp.Matrix([
    [sp.cos(rz_sym), sp.sin(rz_sym), 0],
    [-sp.sin(rz_sym), sp.cos(rz_sym), 0],
    [0, 0, 1]
])

# --- Combined symbolic rotation matrix ---
R_sym = Rz_sym * Ry_sym * Rx_sym
R_numerical = R_sym.subs({rx_sym: rx, ry_sym: ry, rz_sym: rz})
R = np.array(R_numerical.evalf()).astype(np.float64)

# --- Define symbolic variables for parametric rotation matrix ---
a1, a2, a3 = sp.symbols('a1 a2 a3')
b1, b2, b3 = sp.symbols('b1 b2 b3')
c1, c2, c3 = sp.symbols('c1 c2 c3')

R_sym_symbols = R_sym.subs({
    R_sym[0, 0]: a1, R_sym[0, 1]: b1, R_sym[0, 2]: c1,
    R_sym[1, 0]: a2, R_sym[1, 1]: b2, R_sym[1, 2]: c2,
    R_sym[2, 0]: a3, R_sym[2, 1]: b3, R_sym[2, 2]: c3
})

# --- Define translation vector ---
T = np.array([[dx], [dy], [dz]])

# --- Generate base spherical coordinates (shared for all variants) ---
P_base, theta_base, phi_base = uniform_spherical(config)

# --- Define generator functions with shared angular base ---
generators = [
    ("ideal", lambda cfg: (P_base, theta_base, phi_base)),
    ("with_noise", lambda cfg: uniform_spherical_with_noise(cfg, theta=theta_base, phi=phi_base)),
    ("with_geodetic_distortion", lambda cfg: uniform_spherical_with_geodetic_distortion(cfg, theta=theta_base, phi=phi_base))
]

# --- (Optional) prepend real-data generator if INPUT_CFG requests it ---
if INPUT_CFG.get("source", "synthetic").lower() != "synthetic":
    # X_global pochodzi z wcześniejszej komórki: load_points_from_source(...)
    # Zwracamy (3, N) żeby interfejs był taki sam jak w wariantach syntetycznych
    generators = [("real_input", lambda cfg: (X_global.T, None, None))] + generators

# --- Prepare dictionary to store transformed point sets ---
transformed_sets = {}

# --- Iterate through each generator and apply transformation ---
for label, generator in generators:
    print(f"\n\033[1m=== Transforming points: {label} ===\033[0m")

    result = generator(config)

    if len(result) == 3:
        points, theta, phi = result
    else:
        points, theta, phi, _ = result

    # Ensure shape (3, N)
    P = points if points.shape[0] == 3 else points.T

    # Apply transformation
    PP = R @ P + T

    # Optional post-transform noise in the body frame ---
    sigma = float(config.get("gaussian_std", 0.0))  # [m]
    bias  = float(config.get("bias", 0.0))          # [m]

    if (sigma != 0.0 or bias != 0.0) and label != "ideal":
        noise = np.random.normal(loc=0.0, scale=sigma, size=PP.shape)   
        bias_vec = np.array([[bias], [bias], [bias]], dtype=float)      
        PP = PP + noise + bias_vec
 
    # Save to dictionary
    transformed_sets[label] = {
        "P_original": P,
        "P_transformed": PP,
        "theta": theta,
        "phi": phi
    }

    # Print first 3 transformed coordinates
    print("\033[1mTransformed coordinates (first 3 points):\033[0m")
    print(PP[:, :3].T)

# --- Print matrix and angle information ---
print("\033[1m\nRotation around X-axis (radians):\033[0m", rx)
print("\033[1m\nRotation around Y-axis (radians):\033[0m", ry)
print("\033[1m\nRotation around Z-axis (radians):\033[0m", rz)

print("\033[1m\nSymbolic rotation matrix:\033[0m")
print(sp.pretty(R_sym, use_unicode=False, wrap_line=False))

print("\033[1m\nNumerical rotation matrix for configured angles:\033[0m")
print(sp.pretty(R_numerical.evalf(), use_unicode=False, wrap_line=False))

print("\033[1m\nParametric matrix R:\033[0m")
sp.pprint(R_sym_symbols, use_unicode=False)

print("\033[1m\nCoefficients of rotation matrix: a1, a2, ..., c3\033[0m")


### <span style="color:darkgreen; font-weight:bold">Export to Excel – Fixed and Local Frames</span>

At this stage, the generated coordinates (e.g., ideal, noisy, or geodetically distorted points)  
are exported to `.xlsx` files for external analysis or use in downstream transformation algorithms.

Each coordinate set is saved as a table with `X`, `Y`, and `Z` columns, optionally accompanied by point indices.

The export uses the **OpenPyXL** engine to ensure compatibility with spreadsheet software and preserves the structure of the point cloud.

To change the output filenames or worksheet names, adjust the parameters in the code cell below.


In [ ]:
# ====================================================================
# Optional new code for more detailed export with metadata and angles
# ====================================================================


# === Export (Excel + JSON) ===
export_dir = "exports_excel"
os.makedirs(export_dir, exist_ok=True)

def _angles_df(theta, phi):
    """Return a DataFrame with angles if available; otherwise an empty DataFrame."""
    if theta is None or phi is None:
        return pd.DataFrame()
    return pd.DataFrame({"theta_rad": np.asarray(theta), "phi_rad": np.asarray(phi)})

# Helpful suffix: synthetic/pcd/csv
src_tag = INPUT_CFG.get("source", "synthetic").lower()

for label, data in transformed_sets.items():
    # Build DataFrames (ensure (N,3))
    df_original   = pd.DataFrame(data["P_original"].T,    columns=["X", "Y", "Z"])
    df_transformed= pd.DataFrame(data["P_transformed"].T, columns=["X", "Y", "Z"])
    df_angles     = _angles_df(data.get("theta"), data.get("phi"))

    # Metadata sheet – minimal but useful
    meta_rows = [
        ("input_source", src_tag),
        ("input_path", INPUT_CFG.get("path", "")),
        ("n_points", len(df_original)),
        ("roll_grad",  config["roll"]),
        ("pitch_grad", config["pitch"]),
        ("yaw_grad",   config["yaw"]),
        ("dx_m", config["dx"]), ("dy_m", config["dy"]), ("dz_m", config["dz"]),
        ("gaussian_std_m", config.get("gaussian_std", 0.0)),
        ("bias_m", config.get("bias", 0.0)),
    ]
    df_meta = pd.DataFrame(meta_rows, columns=["key", "value"])

    # File name: <source>_<label>_coordinates.xlsx
    fname = f"{src_tag}_{label}_coordinates.xlsx"
    file_path = os.path.join(export_dir, fname)

    with pd.ExcelWriter(file_path, engine="openpyxl") as writer:
        df_original.to_excel(writer, index=False, sheet_name="Original_Global")
        df_transformed.to_excel(writer, index=False, sheet_name="Transformed_Local")
        if not df_angles.empty:
            df_angles.to_excel(writer, index=False, sheet_name="Angles_(rad)")
        df_meta.to_excel(writer, index=False, sheet_name="Metadata")

# Export combined configuration as JSON (config + INPUT_CFG)
config_path = os.path.join(export_dir, "config_used.json")
with open(config_path, "w") as f:
    json.dump({"config": config, "input_cfg": INPUT_CFG}, f, indent=4)

# List exported files for convenience
output_files = sorted(os.listdir(export_dir))
output_files

### <span style="color:darkgreen; font-weight:bold">Visualization and Statistics – Synthetic and Real Inputs</span>

This section provides two complementary views:

**A. Global vs Body (illustrative geometry)**  
Side-by-side 3D plots show the same points in the global (original) and body (transformed) frames.  
This view is **not an uncertainty plot**; it illustrates the rigid-body transformation (R, t) applied to the data.  
Works for both *synthetic* and *real (PCD/CSV)* inputs.

**B. Structure & Displacement summary**  
2D projections (XY/XZ/YZ), a PCA density view, and displacement statistics.  
By default we show **rigid displacement** between global and body coordinates (|Δ| = ‖P_body − P_global‖).  
Optionally, this block can switch to **noise-only** diagnostics (|Δ| between a *noisy* and a *clean* body-frame set),  
if both variants are available.

All figures are saved as **vector graphics (SVG)** for publication-quality output.  
Rendering options (marker size, transparency, grid) can be adjusted inline in the plotting helpers.

In [ ]:
# === Visualization: Global vs. Body Frame (illustrative) ===
# Purpose:
#   - Visual check of spatial structure before/after the rigid-body transform (R, t).
#   - This is NOT an uncertainty plot; it does not show noise, only geometry.
# Works with:
#   - Synthetic sets (e.g., "ideal").
#   - Real input ("real_input") if loaded via INPUT_CFG.
# Output:
#   - Side-by-side 3D scatter + reference wireframe sphere.
#   - Saved as SVG in exports_img/.
    
os.makedirs("exports_img", exist_ok=True)

def plot_on_ax(ax, points, title, color):
    """
    Scatter 3D points plus a matching-radius wireframe sphere on the given Axes3D.
    """
    x, y, z = points
    r = np.max(np.linalg.norm(points, axis=0))  # approximate radius
    u = np.linspace(0, 2*np.pi, 40)
    v = np.linspace(0, np.pi, 40)
    xw = r * np.outer(np.cos(u), np.sin(v))
    yw = r * np.outer(np.sin(u), np.sin(v))
    zw = r * np.outer(np.ones_like(u), np.cos(v))
    ax.scatter(x, y, z, s=6, alpha=0.7, color=color)
    ax.plot_wireframe(xw, yw, zw, color="gray", linewidth=0.6, alpha=0.3)
    ax.set_title(title, pad=10)
    ax.set_xlabel("X [m]")
    ax.set_ylabel("Y [m]")
    ax.set_zlabel("Z [m]")
    ax.set_box_aspect((1,1,1))

# Select which labels to plot
labels_to_plot = ["ideal"]  # default synthetic
if "real_input" in transformed_sets:
    labels_to_plot.insert(0, "real_input")  # show real-data first if present

for lbl in labels_to_plot:
    fig = plt.figure(figsize=(14, 6))
    src = INPUT_CFG.get("source", "synthetic").lower()
    npts = transformed_sets[lbl]["P_original"].shape[1]

    # Global/original
    ax1 = fig.add_subplot(1,2,1, projection="3d")
    plot_on_ax(ax1,
               transformed_sets[lbl]["P_original"],
               title=f"{lbl} – Original (Global Frame)\n{src}, N={npts}",
               color="royalblue")

    # Transformed/body
    ax2 = fig.add_subplot(1,2,2, projection="3d")
    plot_on_ax(ax2,
               transformed_sets[lbl]["P_transformed"],
               title=f"{lbl} – Transformed (Body Frame)",
               color="darkred")

    plt.tight_layout()

    # save as vector format (publication quality)
    out_path = os.path.join("exports_img", f"comparison_{lbl}.svg")
    plt.savefig(out_path, bbox_inches="tight")
    plt.show()

    print(f"Saved: {out_path}")

In [ ]:
# === Alternative visualizations: structure & displacement ===
# Purpose:
#   - Summarize the point-cloud structure (2D projections, PCA) and displacement statistics.
# Default behavior (VIS_MODE = "rigid"):
#   - Displacement is rigid: Δ = P_body - P_global (illustrates the net effect of R, t).
# Optional behavior (VIS_MODE = "noise-only"):
#   - If you have a clean/noisy pair in the body frame (e.g., "ideal" vs "with_geodetic_distortion"
#     or "real_input_clean" vs "real_input_noisy"), set VIS_MODE="noise-only" and provide labels below.
#   - Then Δ = P_body_noisy - P_body_clean (true uncertainty/noise diagnostics).
# Outputs:
#   - 2D projections (XY/XZ/YZ) with equal aspect.
#   - PCA-based density (hexbin).
#   - Histograms of Δ and |Δ|, and |Δ| vs range r (EDM-friendly).
#   - All figures saved as SVG in exports_img/.

os.makedirs("exports_img", exist_ok=True)

def _as_Nx3(P_3xN):
    return P_3xN.T if P_3xN.shape[0] == 3 else P_3xN

def _equal_axes(ax):
    ax.set_aspect('equal', adjustable='box')

# --- NEW: choose visualization mode ---
VIS_MODE = "rigid"  # "rigid" | "noise-only"
NOISE_CLEAN_LABEL = "ideal"                 # e.g., "ideal" or "real_input_clean"
NOISE_NOISY_LABEL = "with_geodetic_distortion"   # e.g., "with_noise" or "real_input_noisy"

# --- Select base label (used for titles and structure plots) ---
label = "real_input" if "real_input" in transformed_sets else "ideal"
src_tag = INPUT_CFG.get("source", "synthetic").lower()

# --- Build P0, P1 depending on mode ---
if VIS_MODE == "noise-only" and NOISE_CLEAN_LABEL in transformed_sets and NOISE_NOISY_LABEL in transformed_sets:
    # noise-only in BODY frame
    P0 = _as_Nx3(transformed_sets[NOISE_CLEAN_LABEL]["P_transformed"])   # (N,3) clean body
    P1 = _as_Nx3(transformed_sets[NOISE_NOISY_LABEL]["P_transformed"])   # (N,3) noisy body
    label_for_titles = f"{NOISE_CLEAN_LABEL} vs {NOISE_NOISY_LABEL}"
else:
    # rigid displacement: GLOBAL vs BODY
    P0 = _as_Nx3(transformed_sets[label]["P_original"])      # (N,3) global
    P1 = _as_Nx3(transformed_sets[label]["P_transformed"])   # (N,3) body
    label_for_titles = f"{label} (rigid)"

# 2) 2D projections (XY, XZ, YZ)
projs = [("XY", 0, 1), ("XZ", 0, 2), ("YZ", 1, 2)]
for name, i, j in projs:
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    for ax, P, ttl in [(axes[0], P0, f"{label_for_titles} – Left ({name})"),
                       (axes[1], P1, f"{label_for_titles} – Right ({name})")]:
        ax.scatter(P[:, i], P[:, j], s=2, alpha=0.5)
        ax.set_xlabel(f"{['X','Y','Z'][i]} [m]")
        ax.set_ylabel(f"{['X','Y','Z'][j]} [m]")
        ax.set_title(ttl); _equal_axes(ax); ax.grid(True, alpha=0.3)
    plt.tight_layout()
    outp = os.path.join("exports_img", f"proj_{name.lower()}_{label}_{src_tag}_{VIS_MODE}.svg")
    plt.savefig(outp, bbox_inches="tight"); plt.show(); print("Saved:", outp)

# 3) PCA density view (same basis for left/right)
X = P0 - P0.mean(axis=0, keepdims=True)
U, S, Vt = np.linalg.svd(X, full_matrices=False)
P0_pca2 = X @ Vt.T[:, :2]
Y = P1 - P1.mean(axis=0, keepdims=True)
P1_pca2 = Y @ Vt.T[:, :2]

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
for ax, P2, ttl in [(axes[0], P0_pca2, f"{label_for_titles} – Left (PCA)"),
                    (axes[1], P1_pca2, f"{label_for_titles} – Right (PCA)")]:
    hb = ax.hexbin(P2[:,0], P2[:,1], gridsize=80, bins='log')
    ax.set_xlabel("PC1 [arb.]"); ax.set_ylabel("PC2 [arb.]")
    ax.set_title(ttl); ax.grid(True, alpha=0.2)
plt.tight_layout()
outp = os.path.join("exports_img", f"pca_hexbin_{label}_{src_tag}_{VIS_MODE}.svg")
plt.savefig(outp, bbox_inches="tight"); plt.show(); print("Saved:", outp)

# 4) Displacement statistics
disp = np.linalg.norm(P1 - P0, axis=1)
fig = plt.figure(figsize=(6,4))
plt.hist(disp, bins=60)
plt.xlabel("|Δ| [m]"); plt.ylabel("count")
plt.title(f"Displacement magnitude – {label_for_titles} ({src_tag}, {VIS_MODE})")
outp = os.path.join("exports_img", f"disp_hist_{label}_{src_tag}_{VIS_MODE}.svg")
plt.savefig(outp, bbox_inches="tight"); plt.show(); print("Saved:", outp)

fig = plt.figure(figsize=(6,4))
for k, axlbl in enumerate(["ΔX","ΔY","ΔZ"]):
    plt.hist(P1[:,k]-P0[:,k], bins=60, histtype='step', label=axlbl)
plt.xlabel("Δ [m]"); plt.ylabel("count"); plt.legend()
plt.title(f"Per-axis displacements – {label_for_titles} ({src_tag}, {VIS_MODE})")
outp = os.path.join("exports_img", f"disp_axes_{label}_{src_tag}_{VIS_MODE}.svg")
plt.savefig(outp, bbox_inches="tight"); plt.show(); print("Saved:", outp)

# 5) |Δ| vs range r (EDM-friendly)
r = np.linalg.norm(P1 if VIS_MODE=="rigid" else P0, axis=1)  # for noise-only, range of the clean body set
fig = plt.figure(figsize=(6,4))
plt.scatter(r, disp, s=3, alpha=0.3)
plt.xlabel("range r [m]"); plt.ylabel("|Δ| [m]")
plt.title(f"|Δ| vs r – {label_for_titles} ({src_tag}, {VIS_MODE})")
plt.tight_layout()
outp = os.path.join("exports_img", f"disp_vs_r_{label}_{src_tag}_{VIS_MODE}.svg")
plt.savefig(outp, bbox_inches="tight"); plt.show(); print("Saved:", outp)

In [ ]:
# === Noise-only diagnostics in the body frame (EDM/Gaussian) ===
# Compare noisy vs. noiseless transform to isolate error model effects.

import numpy as np
import matplotlib.pyplot as plt
import os
os.makedirs("exports_img", exist_ok=True)

# We assume you have at least the "ideal" and one noisy set, e.g. "with_geodetic_distortion"
if "ideal" in transformed_sets and "with_geodetic_distortion" in transformed_sets:
    P_body_clean = transformed_sets["ideal"]["P_transformed"].T      # (N,3)
    P_body_noisy = transformed_sets["with_geodetic_distortion"]["P_transformed"].T  # (N,3)

    disp_noise = np.linalg.norm(P_body_noisy - P_body_clean, axis=1)   # |Δ_noise|
    r_body     = np.linalg.norm(P_body_clean, axis=1)                  # range after R,t

    # 1) Histogram |Δ| from noise only
    plt.figure(figsize=(6,4))
    plt.hist(disp_noise, bins=60)
    plt.xlabel("|Δ| [m]"); plt.ylabel("count")
    plt.title("Noise-only displacement (body frame)\nwith_geodetic_distortion vs ideal")
    plt.tight_layout()
    plt.savefig("exports_img/noise_only_hist.svg", bbox_inches="tight"); plt.show()

    # 2) |Δ| vs r (EDM should give ~linear trend: |Δ| ≈ |a + b r|)
    plt.figure(figsize=(6,4))
    plt.scatter(r_body, disp_noise, s=3, alpha=0.3)
    plt.xlabel("range r (body) [m]"); plt.ylabel("|Δ| [m]")
    plt.title("|Δ| vs r – noise-only (body frame)")
    plt.tight_layout()
    plt.savefig("exports_img/noise_only_disp_vs_r.svg", bbox_inches="tight"); plt.show()

    # Optional: overlay least-squares line for visual guidance
    A = np.vstack([r_body, np.ones_like(r_body)]).T
    k, c = np.linalg.lstsq(A, disp_noise, rcond=None)[0]  # disp ≈ k*r + c
    print(f"Least-squares fit: |Δ| ≈ {c:.6f} + {k:.6e}·r   (compare to EDM a, b)")
else:
    print("Need both 'ideal' and 'with_geodetic_distortion' in transformed_sets for noise-only plots.")

### <span style="color:green; font-weight:bold">Coordinate Uncertainty – Noise and Distortion Only</span>

This section isolates the uncertainty component by comparing **ideal transformed points** 
(without any perturbations) against their **noisy/distorted counterparts** after the same 
rigid-body transformation (R, t). 

Key points:
- The rigid transformation (rotation + translation) is applied identically to both sets, 
  so only **random noise and geodetic distortion effects** remain in the differences.
- The plots show:
  - Error bars in the XY, XZ, and YZ planes,
  - Histograms of ΔX and ΔY,
  - A 2D histogram of ΔX vs ΔY.
- This allows for a direct visualization of **measurement noise impact** independent 
  of alignment or frame transformations.

In [ ]:
# === Helpers for uncertainty analysis (works for synthetic and real data) ===
# 1) add_real_noise_pair: create "real_input_clean" / "real_input_noisy" from an existing "real_input"
# 2) noise_only_residuals: return (P_clean, P_noisy, Δx, Δy, Δz, |Δ|, range_clean)

def add_real_noise_pair(transformed_sets, base_label="real_input",
                        gaussian_std=0.01, bias=0.0, edm=None, seed=42):
    """
    Create 'base_label_clean' and 'base_label_noisy' entries in transformed_sets.

    Noise model:
      - Gaussian XYZ in the body frame: N(0, gaussian_std) + bias (vector of identical bias per axis)
      - Optional EDM range perturbation: r' = r + a + b*r  with edm=(a [m], b [unitless])
        (applied after Gaussian+bias; directions are preserved)

    Parameters
    ----------
    transformed_sets : dict  (must contain base_label with keys "P_transformed" and "P_original")
    base_label       : str   ("real_input" by default)
    gaussian_std     : float [m]
    bias             : float [m]
    edm              : tuple (a, b)  or None
    seed             : int
    """
    assert base_label in transformed_sets, f"Missing '{base_label}' in transformed_sets."

    P0 = transformed_sets[base_label]["P_original"]         # (3, N) global (kept for completeness)
    P1 = transformed_sets[base_label]["P_transformed"]      # (3, N) body (clean baseline)
    theta = transformed_sets[base_label].get("theta", None)
    phi   = transformed_sets[base_label].get("phi", None)

    # Clean copy (no additional noise)
    transformed_sets[f"{base_label}_clean"] = {
        "P_original": P0.copy(),
        "P_transformed": P1.copy(),
        "theta": theta, "phi": phi,
    }

    # Noisy copy
    rng = np.random.default_rng(seed)
    Pn = P1.copy()

    if gaussian_std != 0.0 or bias != 0.0:
        noise = rng.normal(0.0, gaussian_std, size=Pn.shape)
        bias_vec = np.array([[bias], [bias], [bias]], dtype=float)
        Pn = Pn + noise + bias_vec

    if edm is not None:
        a, b = edm
        r  = np.linalg.norm(Pn, axis=0, keepdims=True)             # (1, N)
        u  = Pn / np.maximum(r, 1e-12)                             # unit directions
        r2 = r + a + b * r
        Pn = u * r2

    transformed_sets[f"{base_label}_noisy"] = {
        "P_original": P0.copy(),
        "P_transformed": Pn,
        "theta": theta, "phi": phi,
    }

def noise_only_residuals(transformed_sets, clean_label, noisy_label):
    """
    Compute noise-only residuals between two body-frame sets:
        Δ = P_body(noisy) - P_body(clean)

    Returns
    -------
    Pc   : (N, 3) clean body points
    Pn   : (N, 3) noisy body points
    dx,dy,dz : 1D arrays, residuals per axis [m]
    dmag     : 1D array, |Δ| [m]
    r_clean  : 1D array, range of clean points [m]
    """
    Pc = transformed_sets[clean_label]["P_transformed"].T   # (N, 3)
    Pn = transformed_sets[noisy_label]["P_transformed"].T   # (N, 3)
    d  = Pn - Pc
    r  = np.linalg.norm(Pc, axis=1)
    return Pc, Pn, d[:, 0], d[:, 1], d[:, 2], np.linalg.norm(d, axis=1), r

In [ ]:
# === Coordinate Uncertainty (noise-only, Body frame) ===
# Purpose:
#   Isolate the effect of measurement noise / EDM distortion by comparing a CLEAN
#   body-frame set against a NOISY body-frame set produced with the *same* rigid
#   transformation (R, t). The difference (Δ) removes the influence of rotation/translation
#   and leaves only the uncertainty component.
#
# Works with:
#   - Synthetic:        CLEAN="ideal"  vs NOISY="with_geodetic_distortion" (or "with_noise")
#   - Real data (PCD/CSV) after pairing: CLEAN="real_input_clean" vs NOISY="real_input_noisy"
#
# Output:
#   - XY/XZ/YZ scatter with ±|Δ| error bars
#   - Histograms of ΔX and ΔY
#   - 2D histogram (ΔX vs ΔY)
#   - All saved as SVG to exports_img/

os.makedirs("exports_img", exist_ok=True)

# --- Choose which pair you want to analyze (labels must exist in transformed_sets) ---
CLEAN_LABEL = "ideal"                      # e.g. "ideal" or "real_input_clean"
NOISY_LABEL = "with_geodetic_distortion"  # e.g. "with_noise" or "real_input_noisy"

if CLEAN_LABEL in transformed_sets and NOISY_LABEL in transformed_sets:
    # Body-frame coordinates are stored as (3, N); convert to (N, 3) for convenience.
    P_body_clean = transformed_sets[CLEAN_LABEL]["P_transformed"].T   # (N, 3)
    P_body_noisy = transformed_sets[NOISY_LABEL]["P_transformed"].T   # (N, 3)

    # Noise-only residuals: Δ = noisy - clean  (removes the rigid-body component)
    d = P_body_noisy - P_body_clean
    dx, dy, dz = d[:, 0], d[:, 1], d[:, 2]
    err_mag = np.linalg.norm(d, axis=1)

    # Positions (clean body) for plotting error bars
    Xb, Yb, Zb = P_body_clean[:, 0], P_body_clean[:, 1], P_body_clean[:, 2]

    # --- Quick stats (useful in the console / notebook log) ---
    print(f"[uncertainty] Pair: {CLEAN_LABEL} vs {NOISY_LABEL}")
    print(f"  N = {len(err_mag)}")
    print(f"  mean(ΔX, ΔY, ΔZ) = ({dx.mean():.4e}, {dy.mean():.4e}, {dz.mean():.4e}) m")
    print(f"  std(ΔX, ΔY, ΔZ)  = ({dx.std(ddof=1):.4e}, {dy.std(ddof=1):.4e}, {dz.std(ddof=1):.4e}) m")
    print(f"  RMSE(|Δ|) = {np.sqrt((err_mag**2).mean()):.4e} m")

    # --- Plot grid: 2 rows × 3 columns ---
    fig, axes = plt.subplots(2, 3, figsize=(14, 8))
    fig.suptitle('Coordinate Uncertainty Summary (noise-only, Body frame)', fontsize=14, fontweight='bold')

    # a) X–Y with ±|Δ|
    ax = axes[0, 0]
    ax.errorbar(Xb, Yb, xerr=np.abs(dx), yerr=np.abs(dy),
                fmt='o', ms=3, color='darkred', ecolor='slategray', alpha=0.7, label='±|Δ|')
    ax.set_xlabel('X [m]'); ax.set_ylabel('Y [m]')
    ax.set_title('X–Y with Uncertainty'); ax.legend()
    ax.set_aspect('equal', adjustable='box'); ax.grid(True, alpha=0.25)

    # b) X–Z with ±|Δ|
    ax = axes[0, 1]
    ax.errorbar(Xb, Zb, xerr=np.abs(dx), yerr=np.abs(dz),
                fmt='o', ms=3, color='darkred', ecolor='slategray', alpha=0.7, label='±|Δ|')
    ax.set_xlabel('X [m]'); ax.set_ylabel('Z [m]')
    ax.set_title('X–Z with Uncertainty'); ax.legend()
    ax.set_aspect('equal', adjustable='box'); ax.grid(True, alpha=0.25)

    # c) Y–Z with ±|Δ|
    ax = axes[0, 2]
    ax.errorbar(Yb, Zb, xerr=np.abs(dy), yerr=np.abs(dz),
                fmt='o', ms=3, color='darkred', ecolor='slategray', alpha=0.7, label='±|Δ|')
    ax.set_xlabel('Y [m]'); ax.set_ylabel('Z [m]')
    ax.set_title('Y–Z with Uncertainty'); ax.legend()
    ax.set_aspect('equal', adjustable='box'); ax.grid(True, alpha=0.25)

    # d) Histogram of ΔX
    ax = axes[1, 0]
    ax.hist(dx, bins=40, color='steelblue', alpha=0.85, edgecolor='black')
    ax.set_xlabel('ΔX [m]'); ax.set_ylabel('Count')
    ax.set_title('Histogram of ΔX')

    # e) Histogram of ΔY
    ax = axes[1, 1]
    ax.hist(dy, bins=40, color='steelblue', alpha=0.85, edgecolor='black')
    ax.set_xlabel('ΔY [m]'); ax.set_ylabel('Count')
    ax.set_title('Histogram of ΔY')

    # f) 2D histogram (ΔX vs ΔY) — helps to reveal potential correlation between axes
    ax = axes[1, 2]
    h = ax.hist2d(dx, dy, bins=40, cmap='viridis', cmin=1)
    ax.set_xlabel('ΔX [m]'); ax.set_ylabel('ΔY [m]')
    ax.set_title('2D Histogram: ΔX vs ΔY')
    plt.colorbar(h[3], ax=ax, label='Count')

    # Layout + save
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    out_name = f"uncertainty_noise_only_body_{CLEAN_LABEL}_vs_{NOISY_LABEL}.svg"
    out_path = os.path.join("exports_img", out_name)
    plt.savefig(out_path, bbox_inches="tight")
    plt.show()
    print("Saved:", out_path)

else:
    # Helpful guidance if labels are not present
    print("Uncertainty view requires two body-frame variants in `transformed_sets`:")
    print(f"  - CLEAN label (found: {CLEAN_LABEL in transformed_sets}): {CLEAN_LABEL}")
    print(f"  - NOISY label (found: {NOISY_LABEL in transformed_sets}): {NOISY_LABEL}")
    print("Tips:")
    print("  • For synthetic data, generate 'ideal' and a noisy variant "
          "('with_geodetic_distortion' or 'with_noise').")
    print("  • For real data, first create a pair via add_real_noise_pair("
          "base_label='real_input', gaussian_std=..., edm=(A, B*1e-6)).")

In [ ]:
sigma = float(config.get("gaussian_std", 0.0))
bias  = float(config.get("bias", 0.0))
theory_mag = np.sqrt(3)*sigma  # bez EDM i bias
print(f"Expected |Δ| from Gaussian only: ~{theory_mag:.4f} m "
      f"(observed RMSE |Δ| ~{np.sqrt((err_mag**2).mean()):.4f} m)")
print(f"Configured bias: {bias:.4f} m (means: {dx.mean():.4f}, {dy.mean():.4f}, {dz.mean():.4f})")

In [ ]:
# === Coordinate Uncertainty – noise-only (Body frame) ===
# Choose which pair to compare. For synthetic data you likely want:
#   CLEAN_LABEL="ideal", NOISY_LABEL="with_geodetic_distortion" (or "with_noise")
# For real data after add_real_noise_pair(...):
#   CLEAN_LABEL="real_input_clean", NOISY_LABEL="real_input_noisy"

os.makedirs("exports_img", exist_ok=True)

CLEAN_LABEL = "ideal"
NOISY_LABEL = "with_geodetic_distortion"
SAVE_PREFIX = f"uncertainty_{CLEAN_LABEL}_vs_{NOISY_LABEL}"

if CLEAN_LABEL in transformed_sets and NOISY_LABEL in transformed_sets:
    # Compute residuals
    Pc, Pn, dx, dy, dz, dmag, r_clean = noise_only_residuals(
        transformed_sets, CLEAN_LABEL, NOISY_LABEL
    )

    # --- Summary stats to console ---
    import numpy as np
    print(f"[uncertainty] Pair: {CLEAN_LABEL} vs {NOISY_LABEL} | N={len(dmag)}")
    print(f"  mean(ΔX,ΔY,ΔZ) = ({dx.mean():.4e}, {dy.mean():.4e}, {dz.mean():.4e}) m")
    print(f"  std (ΔX,ΔY,ΔZ) = ({dx.std(ddof=1):.4e}, {dy.std(ddof=1):.4e}, {dz.std(ddof=1):.4e}) m")
    print(f"  RMSE(|Δ|)      = {np.sqrt((dmag**2).mean()):.4e} m")

    # --- 1) Per-axis histograms ---
    fig = plt.figure(figsize=(12, 4))
    for k, (arr, title) in enumerate([(dx, "ΔX"), (dy, "ΔY"), (dz, "ΔZ")], start=1):
        ax = fig.add_subplot(1, 3, k)
        ax.hist(arr, bins=60, histtype="stepfilled", alpha=0.85)
        ax.set_xlabel(f"{title} [m]"); 
        ax.set_ylabel("count" if k == 1 else "")
        ax.set_title(title)
        ax.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.savefig(f"exports_img/{SAVE_PREFIX}_axes_hist.svg", bbox_inches="tight")
    plt.show()

    # --- 2) Magnitude CDF (|Δ|) ---
    x = np.sort(dmag); y = np.linspace(0, 1, len(x), endpoint=True)
    plt.figure(figsize=(6, 4))
    plt.plot(x, y)
    plt.xlabel("|Δ| [m]"); plt.ylabel("cumulative probability")
    plt.title("Empirical CDF of |Δ|")
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.savefig(f"exports_img/{SAVE_PREFIX}_cdf.svg", bbox_inches="tight")
    plt.show()

    # --- 3) 2D histogram ΔX vs ΔY (correlation check) ---
    plt.figure(figsize=(6, 4))
    h = plt.hist2d(dx, dy, bins=40, cmap="viridis", cmin=1)
    plt.xlabel("ΔX [m]"); plt.ylabel("ΔY [m]")
    plt.title("2D Histogram: ΔX vs ΔY")
    plt.colorbar(h[3], label="Count")
    plt.tight_layout()
    plt.savefig(f"exports_img/{SAVE_PREFIX}_dx_dy_2dhist.svg", bbox_inches="tight")
    plt.show()

    # --- 4) Box–Whisker for ΔX, ΔY, ΔZ (noise-only) ---
    fig, ax = plt.subplots(figsize=(6, 4))
    ax.boxplot([dx, dy, dz], tick_labels=[r'$\Delta X$', r'$\Delta Y$', r'$\Delta Z$'],
               showfliers=True)
    ax.set_ylabel("Error [m]")
    ax.set_title("Box–Whisker: Per-Axis Error Distribution (noise-only)")
    ax.grid(True, axis="y", alpha=0.25)
    plt.tight_layout()
    plt.savefig(f"exports_img/{SAVE_PREFIX}_box_whisker.svg", bbox_inches="tight")
    plt.show()

    # --- 5) |Δ| vs range r (good for EDM) ---
    plt.figure(figsize=(6, 4))
    plt.scatter(r_clean, dmag, s=6, alpha=0.35)
    plt.xlabel("range r (clean body) [m]"); plt.ylabel("|Δ| [m]")
    plt.title("|Δ| vs r")
    plt.grid(True, alpha=0.25)
    plt.tight_layout()
    plt.savefig(f"exports_img/{SAVE_PREFIX}_disp_vs_r.svg", bbox_inches="tight")
    plt.show()

else:
    print("Uncertainty analysis requires:")
    print(f"  CLEAN_LABEL present? {CLEAN_LABEL in transformed_sets}  -> {CLEAN_LABEL}")
    print(f"  NOISY_LABEL present? {NOISY_LABEL in transformed_sets}  -> {NOISY_LABEL}")
    print("Tip: for real data call add_real_noise_pair(...) first.")

**Real vs Synthetic note.**  
The calls `add_real_noise_pair(...)` and plotting with labels `real_input_clean/noisy` are only relevant when a real dataset was loaded (i.e., `real_input` exists in `transformed_sets`).  
The safe invocation block below:
1) builds the real clean/noisy pair only if `real_input` is present,  
2) otherwise falls back to the synthetic pair (`ideal` vs `with_geodetic_distortion` or `with_noise`),  
3) and produces the same uncertainty plots in both cases.

In [ ]:
# === Helpers to build clean/noisy pairs and to compute noise-only residuals ===

def add_real_noise_pair(transformed_sets, base_label="real_input",
                        gaussian_std=0.01, bias=0.0, edm=None, seed=42):
    """
    Create 'real_input_clean' and 'real_input_noisy' in transformed_sets from an existing 'real_input'.
    - gaussian_std [m], bias [m] are added in body frame to XYZ.
    - edm = (a, b) adds range-only noise: r' = r + a + b*r (applied after Gaussian/bias).
    """
    assert base_label in transformed_sets, f"Missing base label '{base_label}' in transformed_sets."

    P0 = transformed_sets[base_label]["P_original"]      # (3, N)
    P1 = transformed_sets[base_label]["P_transformed"]   # (3, N)
    theta = transformed_sets[base_label].get("theta", None)
    phi   = transformed_sets[base_label].get("phi", None)

    # CLEAN = copy without extra noise
    transformed_sets[f"{base_label}_clean"] = {
        "P_original":   P0.copy(),
        "P_transformed": P1.copy(),
        "theta": theta, "phi": phi
    }

    # NOISY = apply noise in body frame
    rng = np.random.default_rng(seed)
    Pn = P1.copy()

    if gaussian_std != 0.0 or bias != 0.0:
        noise = rng.normal(0.0, gaussian_std, size=Pn.shape)
        bias_vec = np.array([[bias],[bias],[bias]], dtype=float)
        Pn = Pn + noise + bias_vec

    if edm is not None:
        a, b = edm  # [m], [unitless]
        r  = np.linalg.norm(Pn, axis=0, keepdims=True)          # (1, N)
        u  = Pn / np.maximum(r, 1e-12)                          # directions
        r2 = r + a + b * r
        Pn = u * r2

    transformed_sets[f"{base_label}_noisy"] = {
        "P_original":   P0.copy(),
        "P_transformed": Pn,
        "theta": theta, "phi": phi
    }

def noise_only_residuals(transformed_sets, clean_label, noisy_label):
    """Return (P_clean, P_noisy, Δx, Δy, Δz, |Δ|, range_clean)."""
    Pc = transformed_sets[clean_label]["P_transformed"].T   # (N,3)
    Pn = transformed_sets[noisy_label]["P_transformed"].T   # (N,3)
    d  = Pn - Pc
    r  = np.linalg.norm(Pc, axis=1)
    return Pc, Pn, d[:,0], d[:,1], d[:,2], np.linalg.norm(d, axis=1), r

In [ ]:
# === Universal noise-only plotting suite (works for synthetic and real) ===


def plot_uncertainty_suite(transformed_sets, clean_label, noisy_label,
                           prefix="uncertainty", subsample=None, seed=42):
    """
    Produce: per-axis hist, magnitude CDF, QQ for ΔX, |Δ| vs r (EDM), and a light 2D projection.
    Saves SVGs to exports_img/ with given prefix.
    """
    Pc, Pn, dx, dy, dz, dmag, r = noise_only_residuals(transformed_sets, clean_label, noisy_label)

    if not _HAS_SCIPY:
        print("[info] SciPy not found → skipping Q–Q.")

    # Optional subsampling for dense clouds
    if subsample:
        rng = np.random.default_rng(seed)
        idx = rng.choice(len(dmag), size=min(subsample, len(dmag)), replace=False)
        Pc, dx, dy, dz, dmag, r = Pc[idx], dx[idx], dy[idx], dz[idx], dmag[idx], r[idx]

    tag = f"{prefix}_{clean_label}_vs_{noisy_label}"

    # 1) Per-axis histograms
    plt.figure(figsize=(10,4))
    for k,(arr,lbl) in enumerate([(dx,"ΔX [m]"), (dy,"ΔY [m]"), (dz,"ΔZ [m]")], start=1):
        plt.subplot(1,3,k); plt.hist(arr, bins=60, histtype='stepfilled', alpha=0.8)
        plt.xlabel(lbl); plt.ylabel("count" if k==1 else ""); 
        plt.title(lbl.replace("[m]",""))
    plt.tight_layout(); plt.savefig(f"exports_img/{tag}_axes_hist.svg", bbox_inches="tight"); plt.show()

    # 2) Magnitude CDF
    x = np.sort(dmag); y = np.linspace(0,1,len(x),endpoint=True)
    plt.figure(figsize=(6,4)); plt.plot(x,y)
    plt.xlabel("|Δ| [m]"); plt.ylabel("cumulative probability"); 
    plt.title("Empirical CDF of |Δ|"); 
    plt.tight_layout(); plt.savefig(f"exports_img/{tag}_cdf.svg", bbox_inches="tight"); plt.show()

    # 3) Q-Q for ΔX (Gaussian check)
    try:
        import scipy.stats as st
        z = (dx - dx.mean()) / (dx.std(ddof=1)+1e-12)

        theo_q, ordered = st.probplot(z, dist="norm", fit=False)

        plt.figure(figsize=(6,4))
        plt.scatter(theo_q, ordered, s=10)
        # 45° reference line
        lim = max(abs(ordered.min()), abs(ordered.max()), abs(theo_q).max())
        plt.plot([-lim, lim], [-lim, lim], 'r-')
        plt.xlabel("Theoretical quantiles"); plt.ylabel("Ordered values")
        plt.title("Q–Q Plot: ΔX vs Normal")
        plt.tight_layout()
        plt.savefig(f"exports_img/{tag}_qq_dx.svg", bbox_inches="tight")
        plt.show()
    except Exception as e:
        print("Skipping Q–Q (scipy not installed?):", e)
        
    # 4) |Δ| vs range r (linear-ish if EDM dominates)
    plt.figure(figsize=(6,4))
    plt.scatter(r, dmag, s=5, alpha=0.4)
    plt.xlabel("range r (body) [m]"); plt.ylabel("|Δ| [m]")
    plt.title("|Δ| vs r")
    plt.tight_layout(); plt.savefig(f"exports_img/{tag}_disp_vs_r.svg", bbox_inches="tight"); plt.show()

    # 5) Simple 2D projection with ±|Δ| errorbars (XY)
    plt.figure(figsize=(6,6))
    plt.errorbar(Pc[:,0], Pc[:,1], xerr=np.abs(dx), yerr=np.abs(dy),
                 fmt='o', ms=2, ecolor='slategray', alpha=0.6)
    plt.xlabel("X [m]"); plt.ylabel("Y [m]"); plt.title("X–Y with ±|Δ|")
    plt.gca().set_aspect('equal', adjustable='box'); plt.grid(True, alpha=0.3)
    plt.tight_layout(); plt.savefig(f"exports_img/{tag}_xy_errorbars.svg", bbox_inches="tight"); plt.show()

    print(f"Saved figures with prefix: exports_img/{tag}_*.svg")

In [ ]:
# === Safe invocations: create real noisy pair if applicable, then plot whichever pair exists ===

def list_available_labels(ts):
    print("[info] available sets:", ", ".join(sorted(ts.keys())))

def detect_noise_pair(ts):
    """
    Preference order:
      1) real_input_clean vs real_input_noisy        (real data with injected noise)
      2) ideal vs with_geodetic_distortion           (synthetic EDM)
      3) ideal vs with_noise                         (synthetic Gaussian)
    """
    if "real_input_clean" in ts and "real_input_noisy" in ts:
        return "real_input_clean", "real_input_noisy"
    if "ideal" in ts and "with_geodetic_distortion" in ts:
        return "ideal", "with_geodetic_distortion"
    if "ideal" in ts and "with_noise" in ts:
        return "ideal", "with_noise"
    return None, None

# 0) pokaż co mamy
list_available_labels(transformed_sets)

# 1) jeśli mamy real_input, zbuduj jego parę clean/noisy (real-only)
if "real_input" in transformed_sets:
    print("[info] 'real_input' detected → building clean/noisy pair for real data.")
    add_real_noise_pair(
        transformed_sets,
        base_label="real_input",
        gaussian_std=0.01,    # <- ustaw parametry wg potrzeb
        bias=0.0,
        edm=(0.005, 7e-6),    # A=5 mm, B=7 ppm (b=7e-6)
        seed=42
    )
else:
    print("[info] No 'real_input' found (likely synthetic-only run). Skipping add_real_noise_pair().")

# 2) wybierz dowolną dostępną parę i narysuj wykresy niepewności
cl, nl = detect_noise_pair(transformed_sets)
if cl is not None:
    print(f"[info] Using noise-only pair: {cl} vs {nl}")
    # gęste chmury real → subsample, syntetyczne → bez subsamplingu
    ss = 20000 if cl.startswith("real_input") else None
    plot_uncertainty_suite(
        transformed_sets,
        clean_label=cl,
        noisy_label=nl,
        prefix="uncertainty",
        subsample=ss
    )
else:
    print("[warn] No clean/noisy pair found.")
    print("      Synthetic: ensure 'ideal' + ('with_geodetic_distortion' or 'with_noise').")
    print("      Real: set INPUT_CFG['source'] to 'pcd'/'csv' (creates 'real_input'), then call add_real_noise_pair(...).")